### Class: Machine Learning <br> Date: 11/02/2018 <br> Mini-Project 2

### Members:
__Thomas Henderson__<br>
__Travis Hills__<br>
__Lucas Rufo__<br>
__Kyle Thomas__<br>

In [14]:
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd
from sklearn.model_selection import train_test_split

# Read in the data using Pandas
data = pd.read_csv("data/bezdekIris.csv")
data.head()
# Separate lables and data
########## The labels used below have been added to the CSV file manually. Using loc and iloc 
########## seemed to fail in getting every sample, as the first was interpreted as the name of the column.

y = data["Class"].copy()
X = data.drop(["SW", "SL", "Class"], axis=1)

# Split the data into training and test sets. 
# By the 80-20 rule, 80% of the data is for training and 20% is reserved for testing.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)


In [ ]:
# Data has been split. 
# There are no null entires.
# Normalize data if needed.
# Train SVM models and compare them

